# Parsing MPU-6050 Gyroscope and Accelerometer Data with Python

## i. Links of Interest
 - [ElectronicCats' Github Repo](https://github.com/ElectronicCats/mpu6050.git)

## 1. Loading the file
- used pandas to read the file

- tacked on '0' to the FIFO data until each entry of FIFO data is 150 characters long

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dir = '../test-data/'

raw_data = pd.read_csv(
    dir + 'successfulsample2.txt',
    delimiter='\t',
    header=None
)

for i in range(len(raw_data[:][4])):
    raw_data[:][4][i] += '0' * (150 - len(raw_data[:][4][i]))

raw_fifo = np.vstack([list(num) for num in raw_data[:][4]])

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


array(['6', '6', '6', ..., '6', '6', '6'], dtype='<U1')

## 2. Bitwise Operations to Extract Quaternion Information


In [7]:
def bitwise_op_quat_32(packet, numrange):
    return np.bitwise_or(
        np.bitwise_or(packet[:,numrange[0]].astype(int) << 24, packet[:,numrange[1]].astype(int) << 16),
        np.bitwise_or(packet[:,numrange[2]].astype(int) << 8, packet[:,numrange[3]].astype(int))
    )

def dmp_get_quaternion_32(data, packet):
    data[:,0] = bitwise_op_quat_32(packet, range(4))
    data[:,1] = bitwise_op_quat_32(packet, range(4,8))
    data[:,2] = bitwise_op_quat_32(packet, range(8,12))
    data[:,3] = bitwise_op_quat_32(packet, range(12,16))
    return data

def dmp_get_quaternion_16(data, packet):
    data[:,0] = np.bitwise_or(packet[:,0] << 8, packet[:,1])
    data[:,1] = np.bitwise_or(packet[:,4] << 8, packet[:,5])
    data[:,2] = np.bitwise_or(packet[:,8] << 8, packet[:,9])
    data[:,3] = np.bitwise_or(packet[:,12] << 8, packet[:13])
    
def dmp_get_quaternion(q, packet):
    qI = np.zeros((len(packet), 4))
    status = dmp_get_quaternion_32(qI, packet)
    return np.divide(qI, 16384.0, where=status==0,out=status)
    
    
dmp_get_quaternion(np.zeros((len(raw_fifo), 4)), raw_fifo)
    

array([[1.00860421e+08, 8.40176680e+07, 1.73027840e+07, 1.00860169e+08],
       [1.00860421e+08, 8.40179280e+07, 1.51388163e+08, 1.17507841e+08],
       [1.00860421e+08, 8.39516190e+07, 1.70416640e+07, 8.42145290e+07],
       ...,
       [1.00860421e+08, 8.39531610e+07, 3.37510400e+07, 3.39486720e+07],
       [1.00860421e+08, 8.39531610e+07, 3.37510400e+07, 3.39486720e+07],
       [1.00860421e+08, 8.39531610e+07, 1.17506050e+08, 1.00861185e+08]])

In [1]:
# dmpGetQuaternion(quats, raw_fifo)
# a=np.array(list('213130000'))
# a[0].astype(int)

In [79]:
[x for x in range(4)]

[0, 1, 2, 3]

array([ True,  True,  True,  True, False, False])